In [1]:
import pandas as pd
import numpy as np
import requests
import json
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from datetime import datetime
from os.path import expanduser
from bizdays import Calendar


import warnings
warnings.filterwarnings('ignore')

In [2]:
calendar_path = expanduser("~").replace('\\', '/') + '/Tower Three/T3 Asset - Documentos/Operacional/BACKOFFICE/INPUTS/Calendario/ANBIMA.cal'
# Calendar
cal = Calendar.load(filename= calendar_path)

# Dates
today = datetime.today().strftime('%Y-%m-%d')
print(today)
todays = today.replace('-','')
last_bizday = str(cal.offset(today, -1)) # Using bizdays
print(last_bizday)
last_bizdays = datetime.strptime(last_bizday, '%Y-%m-%d').strftime('%Y%m%d') # Transformando em data para poder formatar
print(last_bizdays)


2023-02-14
2023-02-13
20230213


In [3]:
path_output = expanduser("~").replace('\\', '/') + '/Tower Three/T3 Asset - Documentos/General/Pessoais/Kevyn/Projetos/B3/mkt_cap/files/'
path_output_rede = expanduser("~").replace('\\', '/') + '/Tower Three/T3 Asset - Documentos\/Operacional/BACKOFFICE/INPUTS/MarketCapB3/'
url = 'https://sistemaswebb3-listados.b3.com.br/marketValueProxy/marketValueCall/GetStockExchangeDaily/eyJsYW5ndWFnZSI6InB0LWJyIiwiY29tcGFueSI6IiIsImtleXdvcmQiOiIiLCJwYWdlTnVtYmVyIjoxLCJwYWdlU2l6ZSI6MTIwfQ=='

r = requests.get(url)
#r.content

In [4]:
from base64 import b64decode
from base64 import b64encode

string = b64decode('eyJsYW5ndWFnZSI6InB0LWJyIiwiY29tcGFueSI6IiIsImtleXdvcmQiOiIiLCJwYWdlTnVtYmVyIjoxLCJwYWdlU2l6ZSI6MTIwfQ==').decode('utf-8')
string

'{"language":"pt-br","company":"","keyword":"","pageNumber":1,"pageSize":120}'

In [5]:
string = b64encode(b'{"name":"","pageNumber":1,"pageSize":9999}').decode()
url_personalizada = 'https://sistemaswebb3-listados.b3.com.br/shareCapitalProxy/ShareCapitalCall/GetList/' + string
string

'eyJuYW1lIjoiIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjk5OTl9'

In [6]:
#C:\Users\KevynMarcelino\Tower Three\T3 Asset - Documentos\Operacional\BACKOFFICE\INPUTS\MarketCapB3

In [7]:
import json
data = pd.json_normalize(json.loads(r.content))
data

,Body,Footer,Header.TimeStamp,Header.Row,Header.Date,Header.Column000,Header.Column001,Header.Column002,Header.Column003,Header.Column004,Header.Column005,Header.Column006
0,"[{'Row': '[00000003]', 'Column000': '*', 'Colu...","[{'Row': '[00000383]', 'Column000': '', 'Colum...",2023-02-14T11:17:55.5317726-03:00,[00000001] / [00000002],2023-02-14T00:00:00,IBOV,IBRX100,Empresa,R$ (Mil),Var.(%) em relação a 31/01/2023,USD (Mil),Var.(%) em relação a 31/01/2023


In [8]:
data_final = pd.to_datetime(data['Header.Date'][0]).strftime('%Y-%m-%d')
print(f'Data do arquivo: {data_final}')
data_final = str(cal.offset(data_final, -1))
data_final = datetime.strptime(data_final, '%Y-%m-%d').strftime('%Y%m%d')
print(f'Data útil anterior: {data_final}')

Data do arquivo: 2023-02-14
Data útil anterior: 20230213


In [9]:
columns = data.iloc[0, 5:].to_list()
columns

['IBOV',
 'IBRX100',
 'Empresa',
 'R$ (Mil)',
 'Var.(%) em relação a 31/01/2023',
 'USD (Mil)',
 'Var.(%) em relação a 31/01/2023']

In [10]:
body =  pd.json_normalize(json.loads(r.content), record_path =['Body'])
body

,Row,Column000,Column001,Column002,Column003,Column004,Column005,Column006
0,[00000003],*,**,3R PETROLEUM,"8.791.663,59","-3,44%","1.696.119,07","-5,00%"
1,[00000004],,,3TENTOS,"6.088.466,18","21,05%","1.174.608,59","19,08%"
2,[00000005],,,ABC BRASIL,"4.236.928,81","-2,34%","817.403,41","-3,93%"
3,[00000006],,,ACO ALTONA,"170.182,50","-6,68%","32.832,21","-8,19%"
4,[00000007],,,AERIS,"888.807,61","-27,04%","171.471,93","-28,23%"
...,...,...,...,...,...,...,...,...
375,[00000378],,,WILSON SONS,"4.329.291,02","-4,47%","835.222,25","-6,02%"
376,[00000379],,,WIZ CO,"1.119.350,97","0,00%","215.949,18","0,00%"
377,[00000380],,,WLM IND COM,"818.934,10","7,84%","157.991,69","6,09%"
378,[00000381],*,**,YDUQS PART,"2.651.982,34","-16,94%","511.629,88","-18,29%"


In [11]:
if body.empty:
    print(f'Arquivo da data {data_final} vazio!')

In [12]:
footer =  pd.json_normalize(json.loads(r.content), record_path =['Footer'])
footer

,Row,Column000,Column001,Column002,Column003,Column004,Column005,Column006
0,[00000383],,,TOTAL (IBOV),"3.504.773.100,52","-3,55%","676.153.316,46","-5,11%"
1,[00000384],,,TOTAL (IBXX),"3.621.769.114,02","-3,56%","698.724.604,34","-5,12%"
2,[00000385],,,TOTAL GERAL 380,"4.151.404.020,15","-3,44%","800.903.657,89","-5,00%"


In [13]:
final = pd.concat([body, footer]).reset_index(drop=True)
final.drop(columns=['Row'], inplace=True)
final.columns = columns
final

,IBOV,IBRX100,Empresa,R$ (Mil),Var.(%) em relação a 31/01/2023,USD (Mil),Var.(%) em relação a 31/01/2023
0,*,**,3R PETROLEUM,"8.791.663,59","-3,44%","1.696.119,07","-5,00%"
1,,,3TENTOS,"6.088.466,18","21,05%","1.174.608,59","19,08%"
2,,,ABC BRASIL,"4.236.928,81","-2,34%","817.403,41","-3,93%"
3,,,ACO ALTONA,"170.182,50","-6,68%","32.832,21","-8,19%"
4,,,AERIS,"888.807,61","-27,04%","171.471,93","-28,23%"
...,...,...,...,...,...,...,...
378,*,**,YDUQS PART,"2.651.982,34","-16,94%","511.629,88","-18,29%"
379,,,ZAMP S.A.,"1.321.706,15","-11,93%","254.988,26","-13,36%"
380,,,TOTAL (IBOV),"3.504.773.100,52","-3,55%","676.153.316,46","-5,11%"
381,,,TOTAL (IBXX),"3.621.769.114,02","-3,56%","698.724.604,34","-5,12%"


In [14]:
final.iloc[:, [3,5]] = final.iloc[:, [3,5]].apply(lambda x: x.str.replace('.','').str.replace(',','.')).astype(float)
final.iloc[:, [4,6]] = final.iloc[:, [4,6]].apply(lambda x: x.str.replace('%','').str.replace(',','.')).astype(float)#.apply(lambda x: x/100)
final


,IBOV,IBRX100,Empresa,R$ (Mil),Var.(%) em relação a 31/01/2023,USD (Mil),Var.(%) em relação a 31/01/2023
0,*,**,3R PETROLEUM,8791663.59,-3.44,1696119.07,-5.0
1,,,3TENTOS,6088466.18,21.05,1174608.59,19.08
2,,,ABC BRASIL,4236928.81,-2.34,817403.41,-3.93
3,,,ACO ALTONA,170182.5,-6.68,32832.21,-8.19
4,,,AERIS,888807.61,-27.04,171471.93,-28.23
...,...,...,...,...,...,...,...
378,*,**,YDUQS PART,2651982.34,-16.94,511629.88,-18.29
379,,,ZAMP S.A.,1321706.15,-11.93,254988.26,-13.36
380,,,TOTAL (IBOV),3504773100.52,-3.55,676153316.46,-5.11
381,,,TOTAL (IBXX),3621769114.02,-3.56,698724604.34,-5.12


In [15]:
final.iloc[:, 3] / 1000

0         8791.66359
1         6088.46618
2         4236.92881
3           170.1825
4          888.80761
           ...      
378       2651.98234
379       1321.70615
380    3504773.10052
381    3621769.11402
382    4151404.02015
Name: R$ (Mil), Length: 383, dtype: object

In [16]:
final.to_excel(f'{path_output}{data_final}_mkt_capB3.xlsx', index=False)
final.to_excel(f'{path_output_rede}{data_final}_mkt_capB3.xlsx', index=False)